# FX Option Pricing

1. QuantLib provides robust tools for pricing FX options, including the ability to calculate Greeks. This can be crucial for managing the risk associated with FX options. For example, the company could use QuantLib to price European or American FX options, calculate their sensitivities, and use these metrics for hedging strategies.

In [1]:
!pip install QuantLib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 19.8 MB/s eta 0:00:00


In [19]:
import QuantLib as ql

# Parameters
spot = 1.1
strike = 1.101
volatility = 0.10
domestic_rate = 0.05
foreign_rate = 0.10
option_type = ql.Option.Call
maturity_date = ql.Date(10, 1, 2022)

# Set up the pricing environment
evaluation_date = ql.Date(3, 1, 2022)
ql.Settings.instance().evaluationDate = evaluation_date

# Create the option
payoff = ql.PlainVanillaPayoff(option_type, strike)
exercise = ql.EuropeanExercise(maturity_date)
fx_option = ql.VanillaOption(payoff, exercise)

# Set up the market data
spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot))
vol_handle = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), volatility, ql.Actual365Fixed()))
domestic_ts = ql.YieldTermStructureHandle(ql.FlatForward(evaluation_date, ql.QuoteHandle(ql.SimpleQuote(domestic_rate)), ql.Actual365Fixed()))
foreign_ts = ql.YieldTermStructureHandle(ql.FlatForward(evaluation_date, ql.QuoteHandle(ql.SimpleQuote(foreign_rate)), ql.Actual365Fixed()))

# Set up the Black-Scholes-Merton process
bsm_process = ql.BlackScholesMertonProcess(spot_handle, foreign_ts, domestic_ts, vol_handle)

# Set the pricing engine
fx_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))

# Calculate and print the option price
option_price = fx_option.NPV()
print(f"The FX option price is: {option_price:.4f}")

# Calculate Greeks
print(f"Delta: {fx_option.delta():.4f}")
print(f"Gamma: {fx_option.gamma():.4f}")
print(f"Vega: {fx_option.vega():.4f}")
print(f"Theta: {fx_option.theta():.4f}")
print(f"Rho: {fx_option.rho():.4f}")

The FX option price is: 0.0051
Delta: 0.4482
Gamma: 25.9255
Vega: 0.0602
Theta: -0.1319
Rho: 0.0094


# FX Swap Rate Bootstrapping


2. QuantLib includes tools for bootstrapping yield curves from FX swap rates, which is essential for valuing and managing FX swaps. The FxSwapRateHelper class can be used to construct these curves.

3. Risk Management and Scenario Analysis
QuantLib can be used to perform scenario analysis and stress testing. By simulating different market conditions and recalculating the values of FX derivatives, a company can assess potential risks and prepare appropriate hedging strategies.
4. Integration with Trading Systems
QuantLib’s open-source nature and flexibility allow it to be integrated into a company’s existing trading systems. This integration can facilitate real-time pricing, risk management, and automated trading strategies.

In [30]:
import QuantLib as ql
import pandas as pd

# Assuming you have already defined 'today', 'tenors', and 'curve'
today = ql.Date().todaysDate()
tenors = [ql.Period(1, ql.Months), ql.Period(3, ql.Months), ql.Period(6, ql.Months), ql.Period(1, ql.Years)]
spot_fx = 1.1
fwd_points = [0.0001, 0.0003, 0.0007, 0.0015]  # Example forward points
fixing_days = 2
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)

# Set up the rate helpers
spot_fx_handle = ql.QuoteHandle(ql.SimpleQuote(spot_fx))
collateral_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(0.05)), ql.Actual365Fixed()))

helpers = ql.RateHelperVector()
for tenor, fwd_point in zip(tenors, fwd_points):
    fwd_point_handle = ql.QuoteHandle(ql.SimpleQuote(fwd_point))
    helper = ql.FxSwapRateHelper(fwd_point_handle, spot_fx_handle, tenor, fixing_days, calendar, ql.ModifiedFollowing, False, True, collateral_curve)
    helpers.push_back(helper)

# Bootstrapping the curve
curve = ql.PiecewiseLinearForward(today, helpers, ql.Actual365Fixed())

# Create lists to store the data
start_dates = []
end_dates = []
forward_rates = []

# Calculate forward rates
for i in range(len(tenors) - 1):
    start_date = today + tenors[i]
    end_date = today + tenors[i+1]
    forward_rate = curve.forwardRate(start_date, end_date, ql.Actual365Fixed(), ql.Continuous).rate()

    start_dates.append(start_date.ISO())
    end_dates.append(end_date.ISO())
    forward_rates.append(forward_rate)

# Create a DataFrame
df = pd.DataFrame({
    'Start Date': start_dates,
    'End Date': end_dates,
    'Forward Rate': forward_rates
})

# Display the first 10 rows of the DataFrame
print("\nForward Rates:")
df.head(10)


Forward Rates:


,Start Date,End Date,Forward Rate
0,2024-09-03,2024-11-03,0.051106
1,2024-11-03,2025-02-03,0.051425
2,2025-02-03,2025-08-03,0.051473


# Daily FX Profit and Loss Calculation:

In [23]:
import QuantLib as ql
import pandas as pd

def calculate_daily_pnl(positions, previous_rates, current_rates):
    pnl = 0
    for currency, amount in positions.items():
        previous_rate = previous_rates[currency]
        current_rate = current_rates[currency]
        pnl += amount * (current_rate - previous_rate)
    return pnl

# Example usage
positions = {'EUR': 1000000, 'GBP': -500000, 'JPY': 100000000}
previous_rates = {'EUR': 1.18, 'GBP': 1.38, 'JPY': 0.0091}
current_rates = {'EUR': 1.19, 'GBP': 1.37, 'JPY': 0.0092}

daily_pnl = calculate_daily_pnl(positions, previous_rates, current_rates)
print(f"Daily P&L: ${daily_pnl:,.2f}")

Daily P&L: $25,000.00


# FX Trade Valuation:

The FX Forward Value of -2,080.91 indicates that the forward contract is currently out of the money for the party holding the long position (the buyer of the foreign currency). Here's an explanation of what this means and its implications:  

Interpretation:  

The negative value indicates that the forward rate is less favorable than the current spot rate, adjusted for interest rate differentials.
If the contract were settled today, the holder would incur a loss of $2,080.91.
Factors contributing to this value:
Interest rate differential: The domestic currency (likely USD) has a higher interest rate than the foreign currency, causing the forward rate to be lower than the spot rate.  

Market movements: The spot exchange rate may have moved unfavorably since the contract was initiated.  
  
Implications for risk management:  

For an importer: If this forward was used to hedge future payables, the negative value suggests that the company has protected itself against currency appreciation. The loss on the forward is offset by more favorable exchange rates for their actual transactions.

For an exporter: If this forward was used to hedge future receivables, the negative value indicates that the company will receive less domestic currency than the current market rate would provide.

Accounting considerations:  

Mark-to-market: This value would be recorded as a liability on the balance sheet under hedge accounting rules.  

P&L impact: Depending on the accounting treatment, this loss might be recognized in the income statement or in other comprehensive income.

In [26]:
import QuantLib as ql

def value_fx_forward(spot_rate, domestic_rate, foreign_rate, notional, maturity_date):
    calculation_date = ql.Date().todaysDate()
    ql.Settings.instance().evaluationDate = calculation_date

    day_count = ql.Actual365Fixed()
    calendar = ql.TARGET()

    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_rate))
    domestic_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, domestic_rate, day_count))
    foreign_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, foreign_rate, day_count))

    # Calculate the forward rate
    time_to_maturity = day_count.yearFraction(calculation_date, maturity_date)
    forward_rate = spot_rate * (domestic_ts.discount(time_to_maturity) / foreign_ts.discount(time_to_maturity))

    # Calculate the present value of the forward contract
    forward_value = notional * (forward_rate - spot_rate) * domestic_ts.discount(time_to_maturity)

    return forward_value

# Example usage
spot_rate = 1.18
domestic_rate = 0.01  # USD rate
foreign_rate = -0.005  # EUR rate
notional = 1000000  # Amount in foreign currency (EUR)
maturity_date = ql.Date(15, 9, 2024)

forward_value = value_fx_forward(spot_rate, domestic_rate, foreign_rate, notional, maturity_date)
print(f"FX Forward Value: ${forward_value:,.2f}")

FX Forward Value: $-2,080.91


# FX Position and Exposure Reporting:

In [31]:
import pandas as pd

def calculate_fx_exposures(positions, current_rates, base_currency='USD'):
    exposures = {}
    total_exposure = 0

    for currency, amount in positions.items():
        if currency != base_currency:
            exposure = amount * current_rates[currency]
            exposures[currency] = exposure
            total_exposure += abs(exposure)
        else:
            exposures[currency] = amount
            total_exposure += abs(amount)

    return exposures, total_exposure

# Example usage
positions = {'USD': 5000000, 'EUR': 1000000, 'GBP': -500000, 'JPY': 100000000}
current_rates = {'EUR': 1.19, 'GBP': 1.37, 'JPY': 0.0092}

exposures, total_exposure = calculate_fx_exposures(positions, current_rates)

# Convert the exposures to a pandas DataFrame
df_exposures = pd.DataFrame(list(exposures.items()), columns=['Currency', 'Exposure'])
df_exposures['Exposure'] = df_exposures['Exposure'].map('${:,.2f}'.format)

# Add the total exposure as a separate row
total_row = pd.DataFrame([['Total', f'${total_exposure:,.2f}']], columns=['Currency', 'Exposure'])
df_exposures = pd.concat([df_exposures, total_row], ignore_index=True)

# Display the first 10 rows of the DataFrame
print("\nFX Exposures:")
df_exposures.head(10)


FX Exposures:


,Currency,Exposure
0,USD,"$5,000,000.00"
1,EUR,"$1,190,000.00"
2,GBP,"$-685,000.00"
3,JPY,"$920,000.00"
4,Total,"$7,795,000.00"


# Performance Reporting:

The performance figure of 1.38% represents the overall return or change in value of the FX portfolio over a specific period. Let's break down what this means and its implications for a FX company:

Interpretation:  

The portfolio has increased in value by 1.38% over the measured period.
This is a positive return, indicating that the FX trading activities have been profitable overall.  

Time Frame:  

Without additional context, we don't know the exact time frame this performance represents. It could be daily, weekly, monthly, or year-to-date performance.
For a typical FX trading operation, this might represent daily performance, but it could also be a longer-term measure.  

Components of Performance:  

Realized gains/losses: Profits or losses from closed FX trades.
Unrealized gains/losses: Changes in the market value of open positions.
Interest rate differentials (carry): Gains or losses from holding currencies with different interest rates.  

Transaction costs: Fees and spreads paid for executing trades.


Benchmarking:  

To fully understand if 1.38% is good or not, it should be compared to:
The company's performance targets, Industry benchmarks, Risk-free rate or other relevant market indices.  


Risk-Adjusted Performance:  

While 1.38% is positive, it's important to consider this return in the context of the risk taken. Metrics like Sharpe ratio or Information ratio might be used to assess risk-adjusted performance.

Implications for Risk Management:  

Positive performance might indicate that risk management strategies are effective. However, it's crucial to ensure that this return wasn't achieved by taking excessive risks.  

Volatility Consideration:  

FX markets can be volatile. A 1.38% return could be significant for a short time frame but might be less impressive over a longer period.  

Reporting and Analysis:  

This figure would typically be included in daily performance reports to management. It would be analyzed alongside other metrics like Value at Risk (VaR), position sizes, and market conditions.  

Strategic Implications:  

Consistent positive performance might support arguments for increasing trading limits or expanding into new currency pairs. It could also influence bonus calculations for traders and risk managers.  

Regulatory Considerations:  

Performance metrics are often part of regulatory reporting requirements.
Regulators might scrutinize how this performance was achieved to ensure compliance with risk management policies.  

Client Impact:  

For a FX company that serves clients, good performance could translate to better FX rates or services for customers.  

Long-term Perspective:  

While 1.38% is positive, it's important to view it as part of a longer-term trend rather than a single data point. Consistent performance over time is generally more valuable than short-term gains.

In [28]:
def calculate_performance(initial_positions, initial_rates, current_positions, current_rates):
    initial_value = sum(amount * initial_rates.get(currency, 1) for currency, amount in initial_positions.items())
    current_value = sum(amount * current_rates.get(currency, 1) for currency, amount in current_positions.items())

    return (current_value - initial_value) / initial_value * 100

# Example usage
initial_positions = {'USD': 5000000, 'EUR': 1000000, 'GBP': -500000, 'JPY': 100000000}
initial_rates = {'EUR': 1.18, 'GBP': 1.38, 'JPY': 0.0091}
current_positions = {'USD': 5100000, 'EUR': 950000, 'GBP': -450000, 'JPY': 95000000}
current_rates = {'EUR': 1.19, 'GBP': 1.37, 'JPY': 0.0092}

performance = calculate_performance(initial_positions, initial_rates, current_positions, current_rates)
print(f"Performance: {performance:.2f}%")

Performance: 1.38%
